In [20]:
# =====================================
# DS3: Marine Pollution Visualization with Folium (Test Map)
# =====================================

import pandas as pd
import geopandas as gpd
import folium

# -------------------------
# 1. Load Data
# -------------------------
events = pd.read_excel(r"D:\workshop 2025\project\datasets\ds3\20180920_Marine_Pollution.xlsx")
geo = gpd.read_file(r"D:\workshop 2025\project\datasets\ds3\ENV_Marine_Pollution_Solomons.geojson")

print("Events sample:")
print(events.head(), "\n")
print("GeoJSON sample:")
print(geo.head(), "\n")


Events sample:
              Countries  Chemicals  General garbage  Metals  \
0        American Samoa        NaN              2.0     5.0   
1          Cook Islands        NaN              4.0     NaN   
2                  Fiji        1.0              6.0     5.0   
3      French Polynesia        NaN              NaN     1.0   
4  International Waters        3.0             16.0    50.0   

   Oil splillages and leakages  Old fishing gear  
0                          NaN               NaN  
1                          NaN               NaN  
2                          NaN              38.0  
3                          NaN               1.0  
4                          NaN              59.0   

GeoJSON sample:
   OBJECTID  Join_Count  TARGET_FID  obs_trip_n  pollution_  pollution1  \
0        21           1          21     12530.0      1822.0       359.0   
1        22           1          22     12530.0      1822.0       360.0   
2        23           1          23     12530.0      1822

In [21]:
# -------------------------
# 2. Preprocess Events
# -------------------------
# Create ImpactLevel based on Severity or fallback
severity_col = "Severity" if "Severity" in events.columns else None
pollutant_col = "Pollutant" if "Pollutant" in events.columns else None

if severity_col:
    def classify_impact(val):
        if val > 70:
            return "High"
        elif val > 30:
            return "Moderate"
        else:
            return "Low"
    events["ImpactLevel"] = events[severity_col].apply(classify_impact)
else:
    events["ImpactLevel"] = "Unknown"

print("Impact Levels:", events["ImpactLevel"].value_counts())


Impact Levels: ImpactLevel
Unknown    20
Name: count, dtype: int64


In [22]:
# -------------------------
# 3. Initialize Map
# -------------------------
center_lat = geo.geometry.centroid.y.mean()
center_lon = geo.geometry.centroid.x.mean()

m_test = folium.Map(location=[center_lat, center_lon], zoom_start=6, tiles="OpenStreetMap")

# -------------------------
# 4. Add GeoJSON Layer (Pollution Areas)
# -------------------------
folium.GeoJson(
    geo,
    name="Marine Pollution Zones",
    style_function=lambda x: {
        "fillColor": "#228B22",  # green shade
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.3,
    },
).add_to(m_test)



C:\Users\User\AppData\Local\Temp\ipykernel_6980\2941623538.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = geo.geometry.centroid.y.mean()
C:\Users\User\AppData\Local\Temp\ipykernel_6980\2941623538.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = geo.geometry.centroid.x.mean()


In [23]:
# -------------------------
# 5. Add Pollution Events using LAT_1 and LONG
# -------------------------
color_map = {"Low": "green", "Moderate": "orange", "High": "red", "Unknown": "gray"}

lat_col = "LAT_1"
lon_col = "LONG"

for _, row in events.iterrows():
    lat = row.get(lat_col)
    lon = row.get(lon_col)
    if pd.notna(lat) and pd.notna(lon):
        folium.Circle(
            location=[lat, lon],
            radius=30000,  # adjust size (in meters)
            color=color_map.get(row["ImpactLevel"], "blue"),
            fill=True,
            fill_opacity=0.2,  # translucent
            popup=f"Impact: {row['ImpactLevel']}<br>Severity: {row.get('Severity','N/A')}"
        ).add_to(m_test)



In [24]:
# -------------------------
# 6. Add Layer Control
# -------------------------
folium.LayerControl().add_to(m_test)

# -------------------------
# 7. Save Map
# -------------------------
m_test.save(r"D:\workshop 2025\project\models\model3\marine_pollution_test_map.html")
print("✅ Test map saved as marine_pollution_test_map.html — open this in your browser.")

✅ Test map saved as marine_pollution_test_map.html — open this in your browser.


In [25]:
geo = geo.to_crs(epsg=4326)
